In [7]:
import fastf1
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import numpy as np
import sympy
from shapely.affinity import translate  
import math


In [2]:
#This is the API data that we pull
session = fastf1.get_session(2023, 'Monza', 'R')
session.load(telemetry=True, laps=True, weather=False)
laps = session.laps
lap_55_1 = laps.pick_driver('55').pick_lap(10).get_telemetry()
f1_api_coords = list(zip(lap_55_1["Y"],lap_55_1["X"]))

req         WARNING 	DEFAULT CACHE ENABLED! (147.03 MB) C:\Users\KIRKE\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Italian Grand Prix - Race [v3.3.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '44', '23', '4', '14', '77', '40', '81', '2', '24', '10', '18', '27', '20', '31', '22']


In [3]:
#This plots the 

In [4]:
import geopandas as gpd
import folium

# Load the GeoDataFrame
monza_track = gpd.read_file("bacinger f1-circuits master circuits/it-1922.geojson")
centroid = monza_track.geometry.centroid.iloc[0]

# Ensure the GeoDataFrame is in WGS84 (lat/lon)
#monza_track = monza_track.to_crs(epsg=4326)

# Convert to a projected CRS suitable for buffering (e.g., UTM)
# Note: Choose the correct UTM zone for your specific data. Here, we assume zone 32N.
# This is something we add manually 
monza_track_projected = monza_track.to_crs(epsg=32632)

# Process the geometry to add width by buffering (buffer distance in meters)
width_in_meters = 5  # Specify the width of the track. Adjust according to needs.
monza_track_buffered = monza_track_projected.copy(deep = True)
monza_track_buffered['geometry'] = monza_track_buffered.geometry.buffer(width_in_meters)

C:\Users\KIRKE\AppData\Local\Temp\ipykernel_1444\798706167.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = monza_track.geometry.centroid.iloc[0]


In [6]:
# Create a folium map centered at the centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=15)

# Add the GeoDataFrame to the map
folium.GeoJson(monza_track_buffered).add_to(m)

# Display the map
#m.save("monza_track_map.html")
m  # This line is optional and just to display the map inline, if your

In [9]:
def coordinate_shift(original_centroid, f1_api_coords):
    """This translates the original relative coordinates into longitude and latitude
    original_centroid is the centroid computed from the downloaded track data
    """
    centroid_lon, centroid_lat = (centroid.x, centroid.y)  


      
    # conversion factors - these are approximations, adjust as necessary  
    # 1 degree of latitude is approximately 111 km, and 1 degree of longitude is approximately 111 km multiplied by the cosine of the latitude  
    km_per_degree_lat = 1 / 111  
    km_per_degree_lon = 1 / (111 * math.cos(math.radians(centroid_lat)))  
    
    # your array of tuples  
    xy_coordinates = f1_api_coords
    
    # convert each tuple in the array  
    lonlat_coordinates = []  
    for y,x in xy_coordinates:  
        lon = centroid_lon + (x / 10000) * km_per_degree_lon  # assuming x, y are in meters  
        lat = centroid_lat + (y / 10000) * km_per_degree_lat  # assuming x, y are in meters  
        lonlat_coordinates.append((lon,lat))  
    


    relative_line = LineString(lonlat_coordinates)
    return relative_line



def shift_centroid(original_centroid, relative_line):
    """This shift the centroid computed"""
    # Calculate the distance to translate in each direction  
    dx = original_centroid.x - relative_line.centroid.x  
    dy = original_centroid.y - relative_line.centroid.y  
    
    # Shift the LineString  
    shifted_line = translate(relative_line, xoff=dx, yoff=dy)  
    return shifted_line

In [10]:
test = coordinate_shift(centroid, f1_api_coords)
shifted_line = shift_centroid(centroid, test)

In [11]:
# Update your GeoDataFrame  
gdf = gpd.GeoDataFrame(geometry=[shifted_line], crs="EPSG:4326")    
new_projected = gdf.to_crs(epsg=32632)  
  

kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="Esri")  
folium.GeoJson(new_projected).add_to(kat)  
kat